# SWCON253 Machine Learning

# Computational Foundations

## Scientific Computing in Python: Introduction to NumPy and Matplotlib

### (Optional) Install watermark package 

Install **watermark** from [watermark](https://github.com/rasbt/watermark/)

In [ ]:
!pip install watermark

In [ ]:
%load_ext watermark
%watermark -v -a "Hyeon Jin" -p numpy,scipy,matplotlib

Author: Hyeon Jin

Python implementation: CPython
Python version       : 3.7.10
IPython version      : 5.5.0

numpy     : 1.19.5
scipy     : 1.4.1
matplotlib: 3.2.2



---

# NumPy Basics

---

## NumPy -- Working with Numerical Arrays

### Introduction to NumPy

This lecture offers a quick tour of the NumPy library for working with multi-dimensional arrays in Python. NumPy (short for Numerical Python) was created by Travis Oliphant in 2005, by merging Numarray into Numeric. Since then, the open source NumPy library has evolved into an essential library for scientific computing in Python and has become a building block of many other scientific libraries, such as SciPy, Scikit-learn, Pandas, and others.
What makes NumPy so particularly attractive to the scientific community is that it provides a convenient Python interface for working with multi-dimensional array data structures efficiently; the NumPy array data structure is also called `ndarray`, which is short for *n*-dimensional array. 

In addition to being mostly implemented in C and using Python as a "glue language," the main reason why NumPy is so efficient for numerical computations is that NumPy arrays use **contiguous blocks of memory** that can be efficiently cached by the CPU. In contrast, Python **lists are arrays of pointers** to objects in random locations in memory, which cannot be easily cached and come with a more expensive memory-look-up. However, the computational efficiency and low-memory footprint come at a cost: NumPy arrays have a fixed size and are homogeneous, which means that all elements must have the same type. Homogenous `ndarray` objects have the advantage that NumPy can carry out operations using efficient C loops and avoid expensive type checks and other overheads of the Python API. While adding and removing elements from the end of a Python list is very efficient, altering the size of a NumPy array is very expensive since it requires to create a new array and carry over the contents of the old array that we want to expand or shrink. 

Besides being more efficient for numerical computations than native Python code, NumPy can also be more elegant and readable due to vectorized operations and broadcasting, which are features that we will explore in this lecture. While this material should be sufficient to follow the code examples in this course.

### Motivation: NumPy is fast!

Here is some motivation before we discuss further details, highlighting why learning about and using NumPy is useful. We take a look at a speed comparison with regular Python code. In particular, we are computing a vector dot product in Python (using lists) and compare it with  NumPy's dot-product function. Mathematically, the dot product between two vectors $\mathbf{x}$ and $\mathbf{w}$ can be written as follows:

$$
z = \sum_i x_i w_i = x_1 \times w_1 + x_2 \times w_2 + ... + x_n \times w_n = \mathbf{x}^\top \mathbf{w} 
$$

First, the Python implementation using a for-loop:

In [ ]:
def python_forloop_list_approach(x, w):
    z = 0.
    for i in range(len(x)):
        z += x[i] * w[i]
    return z


a = [1., 2., 3.] # list
b = [4., 5., 6.] # list

print(python_forloop_list_approach(a, b))

Let us compute the runtime for two larger (1000-element) vectors using IPython's `%timeit` magic function:

In [ ]:
large_a = list(range(1000))
large_b = list(range(1000))


%timeit python_forloop_list_approach(large_a, large_b)

Next, we use the `dot` function/method implemented in NumPy to compute the dot product between two vectors and run `%timeit` afterwards:

In [ ]:
import numpy as np


def numpy_dotproduct_approach(x, w):
    # return np.dot(x, w)    
    # return x @ w  # @ acts as dot function
    return x.dot(w)
    

a = np.array([1., 2., 3.])
b = np.array([4., 5., 6.])

print(numpy_dotproduct_approach(a, b))

In [ ]:
large_a = np.arange(1000)
large_b = np.arange(1000)

%timeit numpy_dotproduct_approach(large_a, large_b)

As we can see, replacing the for-loop with NumPy's `dot` function makes the computation of the vector dot product approximately 100 times faster.

### N-dimensional Arrays

NumPy is built around [`ndarrays`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.ndarray.html) objects, which are high-performance multi-dimensional array data structures. Intuitively, we can think of a one-dimensional NumPy array as a data structure to represent a vector of elements -- you may think of it as a fixed-size Python list where all elements share the same type. Similarly, we can think of a two-dimensional array as a data structure to represent a matrix or a Python list of lists. While NumPy arrays can have up to 32 dimensions if it was compiled without alterations to the source code, we will focus on lower-dimensional arrays for the purpose of illustration in this introduction.

Now, let us get started with NumPy by calling the `array` function to create a two-dimensional NumPy array, consisting of two rows and three columns, from a list of lists:

In [ ]:
import numpy as np

lst = [[1, 2, 3], [4, 5, 6]]
ary2d = np.array(lst)
ary2d
ary2d.shape

(2, 3)

![](./images/numpy-intro/array_1.png)

In [ ]:
ary2d[1,2]

By default, NumPy infers the type of the array upon construction. Since we passed Python integers to the array, we can check type of the `ndarray` object `ary2d` by accessing the `dtype` attribute:

In [ ]:
ary2d.dtype

If we want to construct NumPy arrays of different types, we can pass an argument to the `dtype` parameter of the `array` function, for example `np.float32` to create 32-bit arrays. For a full list of supported data types, please refer to the official [NumPy documentation](https://docs.scipy.org/doc/numpy/user/basics.types.html). Once an array has been constructed, we can downcast or recast its type via the `astype` method as shown in the following example:

In [ ]:
float32_ary = ary2d.astype(np.float32)
float32_ary

In [ ]:
float32_ary.dtype

To return the number of elements in an array, we can use the `size` attribute, as shown below:

In [ ]:
ary2d.size

And the number of dimensions of our array (Intuitively, you may think of *dimensions* as the *rank* of a tensor) can be obtained via the `ndim` attribute:

In [ ]:
ary2d.ndim

If we are interested in the number of elements along each array dimension (in the context of NumPy arrays, we may also refer to them as *axes*), we can access the `shape` attribute as shown below:

In [ ]:
ary2d.shape

The `shape` is always a tuple; in the code example above, the two-dimensional `ary` object has two *rows* and *three* columns, `(2, 3)`, if we think of it as a matrix representation.

Similarly, the `shape` of the one-dimensional array only contains a single value:

In [ ]:
np.array([1, 2, 3]).shape

---

# NumPy Array Construction and Indexing

---

### Array Construction Routines

This section provides a non-comprehensive list of array construction functions. Simple yet useful functions exist to construct arrays containing ones or zeros:

In [ ]:
np.ones((3, 3))

In [ ]:
np.zeros((3, 3))

We can use these functions to create arrays with arbitrary values, e.g., we can create an array containing the values 99 as follows:

In [ ]:
np.zeros((3, 3)) + 99

Creating arrays of ones or zeros can also be useful as placeholder arrays, in cases where we do not want to use the initial values for computations but want to fill it with other values right away. If we do not need the initial values (for instance, `'0.'` or `'1.'`), there is also `numpy.empty`, which follows the same syntax as `numpy.ones` and `np.zeros`. However, instead of filling the array with a particular value, the `empty` function creates the array with non-sensical values from memory. We can think of `zeros` as a function that creates the array via `empty` and then sets all its values to `0.` -- in practice, a difference in speed is not noticeable, though.  

NumPy also comes with functions to create identity matrices and diagonal matrices as `ndarrays` that can be useful in the context of linear algebra -- a topic that we will explore later in this section. 

In [ ]:
np.eye(3)

In [ ]:
np.diag((3, 3, 3))

Lastly, I want to mention two very useful functions for creating sequences of numbers within a specified range, namely, `arange` and `linspace`. NumPy's `arange` function follows the same syntax as Python's `range` objects: If two arguments are provided, the first argument represents the start value and the second value defines the stop value of a half-open interval:

In [ ]:
np.arange(4., 10.)

Notice that `arange` also performs type inference similar to the `array` function. If we only provide a single function argument, the range object treats this number as the endpoint of the interval and starts at 0:

In [ ]:
np.arange(5)

Similar to Python's `range`, a third argument can be provided to define the *step* (the default step size is 1). For example, we can obtain an array of all uneven values between one and ten as follows:

In [ ]:
np.arange(1., 11., 2)

The `linspace` function is especially useful if we want to create a particular number of evenly spaced values in a specified half-open interval:

In [ ]:
np.linspace(0., 1., num=5)

### Array Indexing

In this section, we will go over the basics of retrieving NumPy array elements via different indexing methods. Simple NumPy indexing and slicing works similar to Python lists, which we will demonstrate in the following code snippet, where we retrieve the first element of a one-dimensional array:

In [ ]:
ary = np.array([1, 2, 3])
ary[0]

Also, the same Python semantics apply to slicing operations. The following example shows how to fetch the first two elements in `ary`:

In [ ]:
ary[:2] # equivalent to ary[0:2]

If we work with arrays that have more than one dimension or axis, we separate our indexing or slicing operations by commas as shown in the series of examples below:

In [ ]:
ary = np.array([[1, 2, 3],
                [4, 5, 6]])

ary[0, 0] # upper left

In [ ]:
ary[-1, -1] # lower right

In [ ]:
ary[0, 1] # first row, second column

![](./images/numpy-intro/array_2.png)

In [ ]:
ary[0] # entire first row

In [ ]:
ary[:, 0] # entire first column

In [ ]:
ary[:, :2] # first two columns

In [ ]:
ary[0, 0]

---

# NumPy Array Math and Universal Functions

---

### Array Math and Universal Functions

In the previous sections, you learned how to create NumPy arrays and how to access different elements in an array. It is about time that we introduce one of the core features of NumPy that makes working with `ndarray` so efficient and convenient: vectorization. While we typically use for-loops if we want to perform arithmetic operations on sequence-like objects, NumPy provides vectorized wrappers for performing element-wise operations implicitly via so-called *ufuncs* -- short for universal functions.

As of this writing, there are more than 60 ufuncs available in NumPy; ufuncs are implemented in compiled C code and very fast and efficient compared to vanilla Python. In this section, we will take a look at the most commonly used ufuncs, and I recommend you to check out the [official documentation](https://docs.scipy.org/doc/numpy/reference/ufuncs.html#available-ufuncs) for a complete list.

To provide an example of a simple ufunc for element-wise addition, consider the following example, where we add a scalar (here: 1) to each element in a nested Python list:

In [ ]:
lst = [[1, 2, 3], [4, 5, 6]]

for row_idx, row_val in enumerate(lst):
    for col_idx, col_val in enumerate(row_val):
        lst[row_idx][col_idx] += 1
lst

[[2, 3, 4], [5, 6, 7]]

This for-loop approach is very verbose, and we could achieve the same goal more elegantly using list comprehensions:

In [ ]:
lst = [[1, 2, 3], [4, 5, 6]]
[[cell + 1 for cell in row] for row in lst]

We can accomplish the same using NumPy's ufunc for element-wise scalar addition as shown below:

In [ ]:
ary = np.array([[1, 2, 3], [4, 5, 6]])
ary = np.add(ary, 1)
ary

The ufuncs for basic arithmetic operations are `add`, `subtract`, `divide`, `multiply`, and `exp` (exponential). However, NumPy uses operator overloading so that we can use mathematical operators (`+`, `-`, `/`, `*`, and `**`) directly:

In [ ]:
ary + 1

In [ ]:
ary**2

Above, we have seen examples of *binary* ufuncs, which are ufuncs that take two arguments as an input. In addition, NumPy implements several useful *unary* ufuncs, such as `log` (natural logarithm), `log10` (base-10 logarithm), and `sqrt` (square root).

Often, we want to compute the sum or product of array element along a given axis. For this purpose, we can use a ufunc's `reduce` operation. By default, `reduce` applies an operation along the first axis (`axis=0`). In the case of a two-dimensional array, we can think of the first axis as the rows of a matrix. Thus, adding up elements along rows yields the column sums of that matrix as shown below:

In [ ]:
ary = np.array([[1, 2, 3], 
                [4, 5, 6]])

np.add.reduce(ary) # column sumns

To compute the row sums of the array above, we can specify `axis=1`:

In [ ]:
np.add.reduce(ary, axis=1) # row sums

While it can be more intuitive to use `reduce` as a more general operation, NumPy also provides shorthands for specific operations such as `product` and `sum`. For example, `sum(axis=0)` is equivalent to `add.reduce`:

In [ ]:
ary.sum(axis=0) # column sums

![](./images/numpy-intro/ufunc.png)

As a word of caution, keep in mind that `product` and `sum` both compute the product or sum of the entire array if we do not specify an axis:

In [ ]:
ary.sum()

Other useful unary ufuncs are:
    
- `np.mean` (computes arithmetic average)
- `np.std` (computes the standard deviation)
- `np.var` (computes variance)
- `np.sort` (sorts an array)
- `np.argsort` (returns indices that would sort an array)
- `np.min` (returns the minimum value of an array)
- `np.max` (returns the maximum value of an array)
- `np.argmin` (returns the index of the minimum value)
- `np.argmax` (returns the index of the maximum value)
- `np.array_equal` (checks if two arrays have the same shape and elements)

# Broadcasting

A topic we glanced over in the previous section is broadcasting. Broadcasting allows us to perform vectorized operations between two arrays even if their dimensions do not match by creating implicit multidimensional grids. You already learned about ufuncs in the previous section where we performed element-wise addition between a scalar and a multidimensional array, which is just one example of broadcasting. 


![](./images/numpy-intro/broadcasting-1.png)

Naturally, we can also perform element-wise operations between arrays of equal dimensions:

In [ ]:
ary1 = np.array([1, 2, 3])
ary2 = np.array([4, 5, 6])

ary1 + ary2

In contrast to what we are used from linear algebra, we can also add arrays of different shapes. In the example above, we will add a one-dimensional to a two-dimensional array, where NumPy creates an implicit multidimensional grid from the one-dimensional array `ary1`:

In [ ]:
ary3 = np.array([[4, 5, 6], 
                 [7, 8, 9]])

ary3 + ary1 # similarly, ary1 + ary3

![](./images/numpy-intro/broadcasting-2.png)